# Bac à sable RAG

Ce notebook part du principe que la _vector database_ est déjà prête, c'est-à-dire que les étapes suivantes ont déjà été faites:

<div>
<img src="https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png" width="500"/>
</div>

Nous nous intéressons à celles-ci:

<div>
<img src="https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png" width="500"/>
</div>


In [ ]:
import os

import s3fs

from src.db_building import load_retriever, load_vector_database

## Import de la database et du modèle génératif

### Base de données vectorielle

In [ ]:
from src.config import custom_config
from src.model_building import cache_model_from_hf_hub

EMB_MODEL_NAME = "OrdalieTech/Solon-embeddings-large-0.1"
LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"

hf_token = os.environ["HF_TOKEN"]
s3_token = os.environ["AWS_SESSION_TOKEN"]

cache_model_from_hf_hub(EMB_MODEL_NAME, hf_token=hf_token, s3_token=s3_token)
cache_model_from_hf_hub(LLM_MODEL, hf_token=hf_token, s3_token=s3_token)

DATABASE_RUN_ID = "9c9c411829c947799e3acd3df1564c0b"

# Create a custom confz configuration
config = custom_config(
    defaults={  # These defaults can be overriden with env variables
        "MAX_NEW_TOKENS": 2000,
        "MODEL_TEMPERATURE": 1.0,
        "quantization": True,
        "mlflow_run_id": DATABASE_RUN_ID,
    },
    overrides={  # These values are going to be used no matter what
        "UVICORN_TIMEOUT_KEEP_ALIVE": 0,
        "MAX_NEW_TOKENS": 2000,
        "LLM_MODEL": LLM_MODEL,
        "EMB_MODEL_NAME": EMB_MODEL_NAME,
        "mlflow_run_id": DATABASE_RUN_ID,
    },
)
RETURN_FULL_TEXT = True
DO_SAMPLE = True

CLI_MESSAGE_SEPARATOR = (config.cli_message_separator_length * "-") + " \n"

# Remote file configuration
fs = s3fs.S3FileSystem(endpoint_url=config.s3_endpoint_url)

In [ ]:
db = load_vector_database(filesystem=fs, config=config)

In [ ]:
# f"Nombre de documents dans la vector db: {len(db.get()['documents'])}"

## La chaine tout en un (avec langchain)

In [ ]:
from langchain_community.llms import VLLM

from src.config import MODEL_TO_ARGS

retriever, vectorstore = load_retriever(
    vectorstore=db,
    retriever_params={"search_type": "similarity", "search_kwargs": {"k": 10}},
)


llm = VLLM(model=LLM_MODEL, **MODEL_TO_ARGS.get(LLM_MODEL, {}))

In [ ]:
from src.utils import create_prompt_from_instructions, format_docs

system_instructions = """
Tu es un assistant spécialisé dans la statistique publique.
Tu réponds à des questions concernant les données de l'Insee, l'institut national statistique Français.

Réponds en FRANCAIS UNIQUEMENT. Utilise une mise en forme au format markdown.

En utilisant UNIQUEMENT les informations présentes dans le contexte, réponds de manière argumentée à la question posée.

La réponse doit être développée et citer ses sources (titre et url de la publication) qui sont référencées à la fin.
Cite notamment l'url d'origine de la publication, dans un format markdown.

Cite 5 sources maximum.

Tu n'es pas obligé d'utiliser les sources les moins pertinentes.

Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.

Voici le contexte sur lequel tu dois baser ta réponse :
Contexte: {context}
"""

question_instructions = """
Voici la question à laquelle tu dois répondre :
Question: {question}

Réponse:
"""

prompt = create_prompt_from_instructions(system_instructions, question_instructions)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

In [ ]:
answer_pib = rag_chain.invoke("Quelle est la définition du PIB ?")

In [ ]:
answer_pib

In [ ]:
from IPython.display import Markdown, display

display(Markdown(answer_pib.replace("   ", "")))

In [ ]:
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"]))) | prompt | llm | StrOutputParser()
)

rag_chain_with_source = RunnableParallel({"context": retriever, "question": RunnablePassthrough()}).assign(
    answer=rag_chain_from_docs
)

In [ ]:
for chunk in rag_chain_with_source.stream("Quelle est la définition du PIB ?"):
    print(chunk)

In [ ]:
retriever.invoke("Quelle est la définition du PIB ?")[:5]

In [ ]:
rag_chain.batch(["Quelle est la définition du PIB ?" "Où trouver les nouveaux chiffres du chpimage ?"])

## La chaine décomposée

### Modèle génératif

In [ ]:
import os

from langchain_community.llms import VLLM

MAX_NEW_TOKEN = 8192
TEMPERATURE = 0.2
REP_PENALTY = 1.1
TOP_P = 0.8

hf_token = os.environ["HF_TOKEN"]
s3_token = os.environ["AWS_SESSION_TOKEN"]

# cache_model_from_hf_hub(EMB_MODEL_NAME, hf_token=hf_token, s3_token=s3_token)
# cache_model_from_hf_hub(LLM_MODEL, hf_token=hf_token, s3_token=s3_token)
LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"

llm = VLLM(
    model=LLM_MODEL,
    max_new_tokens=MAX_NEW_TOKEN,
    top_p=TOP_P,
    temperature=TEMPERATURE,
    rep_penalty=REP_PENALTY,
    tokenizer_mode="mistral",
    config_format="mistral",
    load_format="mistral",
    enforce_eager=True,
)

In [ ]:
llm.generate(["La recette de la tarte tatin", "tu fais qupoi ?", "où va le monde"])